In [2]:
from IPython.display import Audio
import numpy as np
import mido
import pandas as pd
import matplotlib.pyplot as plt
import pyaudio as pa
from threading import Thread
from scipy import signal

In [3]:
class Instrument:
    
    notes = {'A' : 27.5, 'A#' : 29.0, 'B' : 30.87, 'Bb' : 29.135, 'C' : 16.35, 'C#' : 17.32, 'D' : 18.35, 'D#' : 19.0, 'E' : 20.6, 'Eb' : 19.445, 'F' : 21.83, 'F#' : 23.12 ,'G' : 24.5, 'G#' : 25.96}

    df = pd.read_csv('notes.csv').set_index('Num')
    
    def __init__(self, *args, **kwargs):
        pass
        
    def load(self, midi): 
        self.midi, point = midi, 0
        self.tempos = np.array([[0], [120]])

        for ev in self.midi.tracks[0]:
            point += ev.time
            if ev.is_meta and ev.type == 'set_tempo':
                if point == 0: self.tempos = np.array([[point], [ev.tempo]])
                else: self.tempos = np.append(self.tempos, [[point], [ev.tempo]], axis = 1)

        self.time_steps = self.tempos[1] / (1e6 * self.midi.ticks_per_beat)
        
    def current_timestep(self, point):
        return self.time_steps[self.tempos[0] <= point][-1]

    def synthesize(self, track : int, fs : float, add = False, nochannels = [], stretch_lim = 150, lowtone = 0):
        
        self.track = self.midi.tracks[track]
        self.fs = fs

        if not hasattr(self, 'sound'): self.sound = np.zeros(int(np.round(self.fs * self.midi.length, 0)))
        elif not add: self.sound[:] = 0
            
        point = real_time = 0
        self.used_off = []
        
        for i, ev in enumerate(self.track):
            real_time += ev.time * self.current_timestep(point)
            point += ev.time
            
            if not ev.is_meta and ev.type == 'note_on' and not ev.channel in nochannels and not i in self.used_off:
                info = self.df.loc[ev.note]
                fnote, octave = self.notes[info['Note']], info['Octave']
            
                d, v = self._find_off(ev.note, i, ev.channel)
                duration = d * self.current_timestep(point) * 1.5
                
                if duration: self._add_note(fnote, octave, duration, max(v, ev.velocity), real_time, stretch_lim, lowtone)
                    
    def _add_note(self, fnote, octave, duration, velocity, real_time, stretch_lim, lowtone):
        nt = self._gen_note(freq = fnote * 2**max(0, octave-lowtone), dur = duration, stretch_lim = stretch_lim)
        nt *= velocity / np.abs(nt).max() * octave
        
        idx = int(np.round((self.fs * real_time), 0))

        if idx + nt.size > self.sound.size:
            self.sound = np.append(self.sound, np.zeros(idx + nt.size - self.sound.size))
                    
        self.sound[idx : idx + nt.size] += nt
    
    def _find_off(self, note, idx, channel):
        tot = 0
        for i, ev in enumerate(self.track[idx + 1:]):
            tot += ev.time
            if not ev.is_meta and ev.type in ['note_on', 'note_off'] and ev.note == note and ev.channel == channel:
                self.used_off.append(i + idx + 1)
                return tot, ev.velocity
        return 0#raise Exception('MIDI error - Note never turned off')

In [11]:
class KSInstrument(Instrument):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def _gen_note(self, freq, dur, stretch_factor, b):
        N = int(np.round(self.fs/freq - .5, 0))
        samples = np.zeros(int(np.round(self.fs * dur, 0)))
        wavetable = (2 * np.random.randint(0, 2, N) - 1).astype(float)
        self._update_mean_matrix(N, b)
        
        for i in range(0, samples.size + N - samples.size%N, N):
            inival = wavetable[-1]
            #stretchs = np.random.binomial(1, 1 / stretch_factor, N)
            #idx = stretchs != 0
            #wavetable[idx] = wavetable.dot(self.mean_matrix[:N, : N][:, idx])
            wavetable = wavetable.dot(self.mean_matrix[:N, : N])
            #expos = np.argwhere(idx).reshape(-1)
            #if expos.size: difs = np.append(expos[0], expos[1:] - expos[:-1])
            #else: difs = 0
            #k = np.count_nonzero(idx)
            #wavetable[idx] *= 2.0**(N - k)
            #wavetable[idx] += inival * (b / 2)**(np.arange(1, N + 1)[idx] + N - k)
            wavetable += inival * b**(i + 1) * (1 / 2)**np.arange(1, N + 1)
            wavetable *= b
            samples[i : i + N] = wavetable[ : min(N, samples.size - i)]
        
        return samples
    
    def _update_mean_matrix(self, N, b):

        dif = 0
        if not hasattr(self, 'mean_matrix'): 
            self.mean_matrix, dif = np.zeros((N, N)), N
        
        elif self.mean_matrix.shape[0] < N:
            dif = N - self.mean_matrix.shape[0]
            self.mean_matrix = np.append(self.mean_matrix, np.zeros((dif, N - dif)), axis = 0)
            self.mean_matrix = np.append(self.mean_matrix, np.zeros((N, dif)), axis = 1)

        for i in range(N - dif, N):
            if i: self.mean_matrix[1 : i + 1, i] = self.mean_matrix[: i, i - 1]
            self.mean_matrix[0, i] = 1 * (1/2)**(i+1)
            
class Guitar(KSInstrument):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def _gen_note(self, freq, dur = 1, b = 1, stretch_lim = 150):
        return super()._gen_note(freq = freq, dur = dur, b = 1, stretch_factor = max(1, freq / stretch_lim))
    
class Harp(KSInstrument):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
    def _gen_note(self, freq, dur = 1, velocity = 50, b = 1, stretch_lim = 150):
        return super()._gen_note(freq = freq, dur = dur, b = -1, stretch_factor = max(1, freq / stretch_lim))
    

In [12]:
class Player:
    
    CHUNKSIZE = 1024
    working = False
    
    
    def __init__(self, *args, **kwargs):
        if not Player.working: 
            Player.pAudio = pa.PyAudio() 
            Player.working = True
        
        self.paused = self.stop_playing = False
        self.th = Thread(target = self._keep_playing)
            
    def load(self, data, fs):
        self.sound = (data / np.abs(data).max()).astype(np.float32)
        self.fs = fs
        self._close_stream()
        
        self.stream = self.pAudio.open(rate = self.fs, format = pa.paFloat32, channels = 1, output = True)
        self.stream.start_stream()
        
    def _keep_playing(self):
        data = self.sound[ : self.CHUNKSIZE].tobytes()
        self.idx += 1
        
        while len(data) and not self.stop_playing:
            if not self.paused:
                self.stream.write(data)
                data = self.sound[self.idx * self.CHUNKSIZE : (self.idx + 1) * self.CHUNKSIZE].tobytes()
                self.idx += 1

        self.stop_playing = False
        
    def play(self, **kwargs): 
        
        if self.paused: 
            self.paused = False
        
        else:
            self.stop_playing = True
            if self.th.is_alive(): self.th.join() 

            self.idx, self.stop_playing = 0, False
            self.th = Thread(target = self._keep_playing)
            self.th.start()
    
    def _close_stream(self):
        self.stop_playing = True
        if self.th.is_alive(): self.th.join()
            
        if hasattr(self, 'stream') and self.stream.is_active(): 
            self.stream.stop_stream()
            self.stream.close()   
        
    def stop(self): 
        self.stop_playing = True
        self.paused = False
    
    def pause(self): 
        self.paused = True
        
    def close(self): 
        self._close_stream()
        self.pAudio.terminate()
        Player.working = False

In [7]:
player = Player()
guitar = Guitar()
harp = Harp()

## Hotel California

In [ ]:
file = mido.MidiFile("Samples/hotelcal2.mid")
guitar.load(midi = file)
guitar.synthesize(fs = 48000, track = 0, lowtone = 1, stretch_lim = 120)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

## Under the Sea

In [ ]:
file2 = mido.MidiFile("Samples/Disney_Themes_-_Under_The_Sea.mid")
guitar.load(midi = file2)
for i in range(1, len(file2.tracks)):
    print(f'Track {i} of {len(file2.tracks) - 1}')
    guitar.synthesize(fs = 48000, track = i, add = i != 1)

player.load(guitar.sound, guitar.fs)
player.play()

## Never Gonna Give You Up

In [ ]:
file3 = mido.MidiFile("Samples/Never-Gonna-Give-You-Up-3.mid")
guitar.load(midi = file3)
guitar.synthesize(fs = 48000, track = 0)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

## Happy Birthday

In [ ]:
file4 = mido.MidiFile("Samples/happy_birthday.mid")
guitar.load(midi = file4)
guitar.synthesize(fs = 48000, track = 1)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

In [ ]:
player.stop()

## Losing My Religion

In [ ]:
file5 = mido.MidiFile("Samples/losing_my_religion.mid")
guitar.load(midi = file5)
guitar.synthesize(fs = 48000, track = 2, lowtone = 1, stretch_lim = 200)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

## Pirates of the Caribbean

In [26]:
file8 = mido.MidiFile("Samples/26593_Hes-a-Pirate.mid")
guitar.load(midi = file8)
guitar.synthesize(fs = 48000, track = 1, stretch_lim = 100, lowtone = 1)
guitar.synthesize(fs = 48000, track = 2, stretch_lim = 300, lowtone = -1, add = True)
print('Synthesized')
player.load(guitar.sound, guitar.fs)
player.play()

Synthesized


In [23]:
file8 = mido.MidiFile("Samples/26593_Hes-a-Pirate.mid")
harp.load(midi = file8)
harp.synthesize(fs = 48000, track = 1, stretch_lim = 100, lowtone = -1)
harp.synthesize(fs = 48000, track = 2, stretch_lim = 300, lowtone = -1, add = True)
print('Synthesized')
player.load(harp.sound, harp.fs)
player.play()

Synthesized


In [27]:
player.pause()

## Sign of the Times

In [ ]:
file9 = mido.MidiFile("Samples/AUD_HTX0815.mid")
guitar.load(midi = file9)
guitar.synthesize(fs = 48000, track = 0, stretch_lim = 120)
player.load(guitar.sound, guitar.fs)
player.play()

## Game of Thrones

In [8]:
file10 = mido.MidiFile("Samples/GameofThrones.mid")
guitar.load(midi = file10)
guitar.synthesize(fs = 48000, track = 1, stretch_lim = 120)
guitar.synthesize(fs = 48000, track = 2, stretch_lim = 120, add = True)
guitar.synthesize(fs = 48000, track = 3, stretch_lim = 120, add = True)
guitar.synthesize(fs = 48000, track = 4, stretch_lim = 120, add = True)
player.load(guitar.sound, guitar.fs)
player.play()

In [9]:
player.pause()

In [10]:
player.play()